In [2]:
import random
import pandas as pd
pd.set_option('display.max_rows', None)

NUMBER_OF_QUESTIONS = 4

# import questions
# print("Questions\n")
questions_file = 'Questions_data_prj3.csv'
questions_df = pd.read_csv(questions_file)
# display(questions_df)

# import answers
# print("Answers\n")
answers_file = 'Answers_data_prj3_update2.csv'
answers_df = pd.read_csv(answers_file)
# display(answers_df)

# add feedback text to each question
answers_df['feedback_text'] = pd.Series(dtype='str')

# mark answer rows as correct or incorrect
answers_df['is_correct'] = pd.Series(dtype='Int64')

for index, row in answers_df.iterrows():
    # setup
    corresponding_question_row = questions_df.loc[questions_df['Question_id'] == row['Question_id']]
    correct_answer = str(corresponding_question_row['Correct_answer_choice'].iloc[0])
    student_answer = str(row['Student_choice_on_question']).replace(" ", "")

    # parse feedback for all options
    feedback_text = ""

    choice_a = corresponding_question_row['Choice_A_text'][row['Question_id'] - 1]
    choice_b = corresponding_question_row['Choice_B_text'][row['Question_id'] - 1]
    choice_c = corresponding_question_row['Choice_C_text'][row['Question_id'] - 1]
    choice_d = corresponding_question_row['Choice_D_text'][row['Question_id'] - 1]

    if "A" in correct_answer:
        feedback_text += choice_a + " "
    if "B" in correct_answer:
        feedback_text += choice_b + " "
    if "C" in correct_answer:
        feedback_text += choice_c + " "
    if "D" in correct_answer:
        feedback_text += choice_d + " "
        
    answers_df.at[index, 'feedback_text'] = feedback_text
    
    # mark answer correct or incorrect
    if (student_answer == correct_answer):
        answers_df.at[index, 'is_correct'] = 1
    else:
        answers_df.at[index, 'is_correct'] = 0

# generate a specific question's answers
def generate_question_answers(answers_df, question_id):
    rows_by_question_id = answers_df[answers_df['Question_id'] == question_id]

    correct_answers = rows_by_question_id[rows_by_question_id['is_correct'] == 1]
    incorrect_answers = rows_by_question_id[rows_by_question_id['is_correct'] == 0]

    # the incorrect answer options should be 3 * 2 usually, if a student wants to retake the problem
    # so they have 3 new options on the second retake
    # however, if there are not enough answers in the incorrect_answers df, then it chooses as many as it can
    number_of_incorrect_answers_to_select = 6
    if len(incorrect_answers) < 6:
        number_of_incorrect_answers_to_select = len(incorrect_answers)

    incorrect_answer_indicies = random.sample(range(0, len(incorrect_answers)), number_of_incorrect_answers_to_select)
    partial_answer_options_array = []

    partial_answer_options_array.append(correct_answers.iloc[random.randrange(0, len(correct_answers) - 1, 1)])
    for index in incorrect_answer_indicies:
        partial_answer_options_array.append(incorrect_answers.iloc[index])

    return partial_answer_options_array

# append results of previous function to answer options array
answer_options_array = []
for i in range(1, NUMBER_OF_QUESTIONS + 1):
    partial_answer_options_array = generate_question_answers(answers_df, i)
    for item in partial_answer_options_array:
        answer_options_array.append(item)
    
answer_options_df = pd.DataFrame(answer_options_array, columns = list(answers_df.columns.values))
display(answer_options_df)

# export to json
answer_options_df.to_json('export_dataframe.json')

# export to csv
# answer_options_df.to_csv('export_dataframe.csv')


,Answer_text,Question_id,Student_score_on_question,Student_choice_on_question,Quiz_score,Average_quizzes_score,feedback_text,is_correct
4,It's not specific enough. Portraiture could be...,1,1.00,A,6.00,6.8,"Rajeev is getting a lot of practice, but he sh...",1
3,"lack of deliberate practice, to say, always pr...",1,0.00,B,6.75,6.7,"Rajeev is getting a lot of practice, but he sh...",0
1,"Even without deliberate practice, Rajeev could...",1,0.00,B,8.50,8.0,"Rajeev is getting a lot of practice, but he sh...",0
0,it's not deliberate practice,1,0.00,B,6.00,7.8,"Rajeev is getting a lot of practice, but he sh...",0
2,The task of creating a portrait is a complex o...,1,0.00,B,8.00,8.1,"Rajeev is getting a lot of practice, but he sh...",0
39,"No, because he doesn't have an external mentor...",2,1.00,"B, C",8.00,7.3,"No, because he is not getting task feedback fr...",1
34,Not receiving immediate feedback may make Raje...,2,0.75,B,4.75,6.7,"No, because he is not getting task feedback fr...",0
26,"Not exactly. To engage in deliberate practice,...",2,0.75,B,6.75,7.2,"No, because he is not getting task feedback fr...",0
28,no.there's no external resource to provide fee...,2,0.75,B,8.75,7.7,"No, because he is not getting task feedback fr...",0
25,"No, because deliberate practice requires that ...",2,0.50,"C, D",8.50,8.0,"No, because he is not getting task feedback fr...",0


In [2]:
# Note: This is scaffolding for a similarity score.

# import nltk

# difficulty ranges from 1-10
# the lower the value, the greater the spread (and higher ease) between the correct answer and incorrect answers)
# difficulty = 1

# ca_sorted_by_quiz_score = correct_answers.sort_values('Quiz_score', ascending=False)
# ia_sorted_by_quiz_score = incorrect_answers.sort_values('Quiz_score', ascending=True)

# incorrect_answer_options = {}
# incorrect_indicies_to_use = []

# for index, row in ia_sorted_by_quiz_score.iterrows():
#     tokenized_answer_text = nltk.word_tokenize(row['Answer_text'])
#     tokenized_answer_text_alpha_only = list([val.lower() for val in tokenized_answer_text if val.isalpha()])
# # import pdb; pdb.set_trace()
#     tokenized_answer_text_uniq = list(set(tokenized_answer_text_alpha_only))
#     tokenized_answer_text_uniq.sort()
#     incorrect_answer_options[index] = tokenized_answer_text_uniq 
# #     print(tokenized_answer_text_uniq)
# # print(incorrect_answer_options)
# # display(ia_sorted_by_quiz_score.iloc[0])
# for i in range(0, len(ia_sorted_by_quiz_score)):
# #     print(incorrect_answer_options[i])
    
#     if (i != 0):
#         print(set(incorrect_answer_options[i]) & set(incorrect_answer_options[i - 1]))